1. Need to go through each on and convert hits for instruments to lists
2. Probably easiest to make sure dataframe length is divisible by 16, if not, delete the number from the tail to make that happen
e.g. In UC321, the last row is "SD 1", so we can remove this
3. Create HVO objects for 32 hits
4. For velocities, we are using the "* Peak" columns. I've noticed some values above 1, so we need to normalize these. However, there are many values that are low, e.g. 0.004, 0.05, etc... How do we manage these? -> just divide by max peak (maybe for column), just to keep things at 1 or below
5. Recordings UC_311, UC_312, UC_321 contain four drummers, the others only have three (UC_211 for example). How do we convert these to four streams? -> just keep as is i.e R2 is just zeros


In [13]:
import csv
import os
import note_seq
import numpy as np
import pretty_midi
import pandas as pd
import random

In [28]:
df = pd.read_csv('/Users/justinbosma/Desktop/TripleStreams/candombe/candombe_annotations/UC_211.csv')

In [30]:
print(df.head(30))

    Take  SD  Cycle  CycleTime  Iso.Time  C_hit  C_vel  C_off  P_hit  P_vel  \
0    211   1      1   3.410476  3.410476    0.0    0.0    0.0    0.0    0.0   
1    211   2      1        NaN  3.562351    0.0    0.0    0.0    0.0    0.0   
2    211   3      1        NaN  3.714226    0.0    0.0    0.0    0.0    0.0   
3    211   4      1        NaN  3.866101    0.0    0.0    0.0    0.0    0.0   
4    211   5      1        NaN  4.017976    0.0    0.0    0.0    0.0    0.0   
5    211   6      1        NaN  4.169851    0.0    0.0    0.0    0.0    0.0   
6    211   7      1        NaN  4.321726    0.0    0.0    0.0    0.0    0.0   
7    211   8      1        NaN  4.473601    0.0    0.0    0.0    0.0    0.0   
8    211   9      1        NaN  4.625476    0.0    0.0    0.0    0.0    0.0   
9    211  10      1        NaN  4.777351    0.0    0.0    0.0    0.0    0.0   
10   211  11      1        NaN  4.929226    0.0    0.0    0.0    0.0    0.0   
11   211  12      1        NaN  5.081101    0.0    0

In [14]:
from hvo_sequence import HVO_Sequence
from hvo_sequence import ROLAND_REDUCED_MAPPING

In [15]:
from pickle import load
hvo_candombe_loaded = load(open("/Users/justinbosma/Desktop/TripleStreams/candombe/output/UC_211.hvo", "rb"))

In [39]:
print(hvo_candombe_loaded.hvo.shape)
print(hvo_candombe_loaded.hits[3])
print(hvo_candombe_loaded.offsets[3])
print(hvo_candombe_loaded.velocities[3] == [0., 0., 0.032734, 0.032734])

(1344, 12)
[0. 0. 1. 1.]
[0.         0.         0.12936171 0.        ]
[ True  True  True  True]


In [44]:
(np.allclose(hvo_candombe_loaded.offsets[3], [0., 0., 0.12936171, 0.]) or np.allclose(hvo_candombe_loaded.offsets[3], [0., 0., 0.12936171, 0.12936171]))

True

In [4]:
Four_Voices = {
        "voice_1": [36],
        "voice_2": [37],
        "voice_3": [38],
        "voice_4": [39]
    }

In [5]:
# create an instance of HVO_Sequence
hvo_seq_uc321 = HVO_Sequence(
    beat_division_factors=[4],                 # select the subdivision of the beat
    drum_mapping=Four_Voices
)

In [6]:
# Add time_signature
hvo_seq_uc321.add_time_signature(time_step=0, numerator=4, denominator=4)

# Add tempo
hvo_seq_uc321.add_tempo(time_step=0, qpm=120)

In [7]:
hvo_length = int(len(df["R1_hit"]) - len(df["R1_hit"])%32)

In [24]:

hvo_seq_uc321.zeros(hvo_length)
hvo_seq_uc321.hvo.shape#, hvo_seq_uc321.hits.shape, hvo_seq_uc321.hits#, hvo_seq.velocities, hvo_seq.offsets,

(1344, 12)

In [17]:
hvo_seq_uc321.hits[0]

array([0., 0., 0., 0.])

In [11]:



for i in range(hvo_length): #int(len(df)/32)):
    print(f"C_hit_{i}___{df['C_hit'][i]}")
    if df['C_hit'][i] > 0:
        hvo_seq_uc321.hits[i][0] = 1
        hvo_seq_uc321.velocities[i][0] = df['C_vel'][i]
        hvo_seq_uc321.offsets[i][0] = df['C_off'][i]
    print(f"P_hit_{i}___{df['P_hit'][i]}")
    if df['P_hit'][i] > 0:
        hvo_seq_uc321.hits[i][1] = 1
        hvo_seq_uc321.velocities[i][1] = df['P_vel'][i]
        hvo_seq_uc321.offsets[i][1] = df['P_off'][i]
    print(f"R1_hit{i}___{df['R1_hit'][i]}")
    if df['R1_hit'][i] > 0:
        hvo_seq_uc321.hits[i][2] = 1
        hvo_seq_uc321.velocities[i][2] = df['R1_vel'][i]
        hvo_seq_uc321.offsets[i][2] = df['R1_off'][i]
    if(df['C_hit'][i] or df['P_hit'][i] or df['R1_hit'][i]) > 0:
        hvo_seq_uc321.hits[i][3] = 1
        hvo_seq_uc321.velocities[i][3] = max(df['C_vel'][i], df['P_vel'][i], df['R1_vel'][i])
        hvo_seq_uc321.offsets[i][3] = random.choice([df['C_off'][i], df['P_off'][i], df['R1_off'][i]])

    




C_hit_0___0.0
P_hit_0___0.0
R1_hit0___0.0
C_hit_1___0.0
P_hit_1___0.0
R1_hit1___0.0
C_hit_2___0.0
P_hit_2___0.0
R1_hit2___0.0
C_hit_3___0.0
P_hit_3___0.0
R1_hit3___1.0
C_hit_4___0.0
P_hit_4___0.0
R1_hit4___0.0
C_hit_5___0.0
P_hit_5___0.0
R1_hit5___0.0
C_hit_6___0.0
P_hit_6___0.0
R1_hit6___1.0
C_hit_7___0.0
P_hit_7___0.0
R1_hit7___0.0
C_hit_8___0.0
P_hit_8___0.0
R1_hit8___0.0
C_hit_9___0.0
P_hit_9___0.0
R1_hit9___0.0
C_hit_10___0.0
P_hit_10___0.0
R1_hit10___1.0
C_hit_11___0.0
P_hit_11___0.0
R1_hit11___0.0
C_hit_12___0.0
P_hit_12___0.0
R1_hit12___1.0
C_hit_13___0.0
P_hit_13___0.0
R1_hit13___0.0
C_hit_14___0.0
P_hit_14___0.0
R1_hit14___0.0
C_hit_15___0.0
P_hit_15___0.0
R1_hit15___0.0
C_hit_16___0.0
P_hit_16___0.0
R1_hit16___1.0
C_hit_17___0.0
P_hit_17___0.0
R1_hit17___0.0
C_hit_18___0.0
P_hit_18___0.0
R1_hit18___0.0
C_hit_19___0.0
P_hit_19___0.0
R1_hit19___1.0
C_hit_20___0.0
P_hit_20___0.0
R1_hit20___0.0
C_hit_21___0.0
P_hit_21___0.0
R1_hit21___0.0
C_hit_22___0.0
P_hit_22___0.0
R1_hit22__

In [12]:
hvo_seq_uc321.velocities


array([[0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      ],
       ...,
       [0.220383, 0.146495, 0.      , 0.220383],
       [0.256687, 0.      , 0.411803, 0.411803],
       [0.596641, 0.295471, 0.431794, 0.596641]])